In [6]:
import os
import itertools

import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.pipeline as pipeline
import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.bestfit_params as bestfit

import importlib
importlib.reload(pipeline)
importlib.reload(setup_bestfit)
importlib.reload(bestfit)

<module 'climatools.lblnew.bestfit_params' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/bestfit_params.py'>

In [15]:
def param_map():
    '''
    Copy-n-pasted from run_lblnew_pipeline.ipynb
    '''
    return {('co2', '3a'): {'atmpro': 'mls',
  'band': '3a',
  'commitnumber': 'd5931a1',
  'conc': 0.0004,
  'dv': 0.001,
  'klin': 6.375563e-24,
  'molecule': 'co2',
  'ng_adju': [0, 0, 0],
  'ng_refs': [3, 2, 4],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(1, 250), (10, 250), (500, 250)],
  'tsfc': 294,
  'vmax': 620,
  'vmin': 540,
  'w_diffuse': [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)],
  'wgt': [(0.7, 0.3, 0.7), (0.7, 0.6), (0.4, 0.5, 0.8, 0.95)]},
 ('co2', '3b'): {'atmpro': 'mls',
  'band': '3b',
  'commitnumber': 'd5931a1',
  'conc': 0.0004,
  'dv': 0.001,
  'klin': 0,
  'molecule': 'co2',
  'ng_adju': [0, 0],
  'ng_refs': [5, 2],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(1, 250), (10, 250)],
  'tsfc': 294,
  'vmax': 720,
  'vmin': 620,
  'w_diffuse': [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)],
  'wgt': [(0.7, 0.8, 0.8, 0.7, 0.5), (0.6, 0)]},
 ('co2', '3c'): {'atmpro': 'mls',
  'band': '3c',
  'commitnumber': 'd5931a1',
  'conc': 0.0004,
  'dv': 0.001,
  'klin': 6.375563e-24,
  'molecule': 'co2',
  'ng_adju': [0, 0, 0],
  'ng_refs': [3, 2, 4],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(1, 250), (10, 250), (500, 250)],
  'tsfc': 294,
  'vmax': 800,
  'vmin': 720,
  'w_diffuse': [(1.7, 1.6, 1.8), (1.8, 1.7), (1.5, 1.6, 1.7, 1.8)],
  'wgt': [(0.6, 0.4, 0.7), (0.7, 0.5), (0.3, 0.4, 0.85, 0.95)]},
 ('co2', '9'): {'atmpro': 'mls',
  'band': '9',
  'commitnumber': 'd5931a1',
  'conc': 0.0004,
  'dv': 0.001,
  'klin': 6.5e-24,
  'molecule': 'co2',
  'ng_adju': [0, 0],
  'ng_refs': [3, 3],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(1, 250), (50, 250)],
  'tsfc': 294,
  'vmax': 3000,
  'vmin': 1900,
  'w_diffuse': [(1.66, 1.66, 1.66), (1.66, 1.66, 1.8)],
  'wgt': [(0.7, 0.7, 0.7), (0.7, 0.7, 0.8)]},
 ('h2o', '1'): {'atmpro': 'mls',
  'band': '1',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 0,
  'molecule': 'h2o',
  'ng_adju': [0, 0, 0],
  'ng_refs': [3, 3, 6],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(1, 250), (30, 250), (300, 250)],
  'tsfc': 294,
  'vmax': 340,
  'vmin': 20,
  'w_diffuse': [(1.66, 1.66, 1.66),
   (1.66, 1.66, 1.66),
   (1.66, 1.66, 1.5, 1.5, 1.5, 1.5)],
  'wgt': [(0.2, 0.2, 0.2),
   (0.2, 0.2, 0.2),
   (0.2, 0.2, 0.65, 0.65, 0.65, 0.65)]},
 ('h2o', '2'): {'atmpro': 'mls',
  'band': '2',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 0,
  'molecule': 'h2o',
  'ng_adju': [-2, -2, 0],
  'ng_refs': [2, 3, 6],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(1, 250), (10, 250), (500, 250)],
  'tsfc': 294,
  'vmax': 540,
  'vmin': 340,
  'w_diffuse': [(1.66, 1.66),
   (1.8, 1.8, 1.8),
   (1.8, 1.66, 1.45, 1.45, 1.45, 1.45)],
  'wgt': [(0.6, 0.6), (0.6, 0.6, 0.6), (0.6, 0.6, 0.6, 0.6, 0.6, 0.6)]},
 ('h2o', '3a'): {'atmpro': 'mls',
  'band': '3a',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 0,
  'molecule': 'h2o',
  'ng_adju': [0, 0],
  'ng_refs': [2, 6],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(10, 250), (600, 250)],
  'tsfc': 294,
  'vmax': 620,
  'vmin': 540,
  'w_diffuse': [(1.9, 1.7), (1.4, 1.4, 1.4, 1.55, 1.6, 1.66)],
  'wgt': [(0.7, 0.7), (0.7, 0.5, 0.5, 0.5, 0.5, 0.5)]},
 ('h2o', '3b'): {'atmpro': 'mls',
  'band': '3b',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 1e-24,
  'molecule': 'h2o',
  'ng_adju': [0],
  'ng_refs': [6],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(600, 250)],
  'tsfc': 294,
  'vmax': 720,
  'vmin': 620,
  'w_diffuse': [(1.66, 1.66, 1.66, 1.66, 1.66, 1.66)],
  'wgt': [(0.8, 0.8, 0.8, 0.6, 0.6, 0.9)]},
 ('h2o', '3c'): {'atmpro': 'mls',
  'band': '3c',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 5e-25,
  'molecule': 'h2o',
  'ng_adju': [0],
  'ng_refs': [5],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(600, 250)],
  'tsfc': 294,
  'vmax': 800,
  'vmin': 720,
  'w_diffuse': [(1.55, 1.6, 1.66, 1.66, 1.8)],
  'wgt': [(0.5, 0.5, 0.6, 0.7, 0.9)]},
 ('h2o', '4'): {'atmpro': 'mls',
  'band': '4',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 1e-24,
  'molecule': 'h2o',
  'ng_adju': [0],
  'ng_refs': [3],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(600, 250)],
  'tsfc': 294,
  'vmax': 980,
  'vmin': 800,
  'w_diffuse': [(1.66, 1.66, 1.85)],
  'wgt': [(0.5, 0.55, 0.85)]},
 ('h2o', '5'): {'atmpro': 'mls',
  'band': '5',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 1e-24,
  'molecule': 'h2o',
  'ng_adju': [0],
  'ng_refs': [3],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(600, 250)],
  'tsfc': 294,
  'vmax': 1100,
  'vmin': 980,
  'w_diffuse': [(1.66, 1.66, 1.8)],
  'wgt': [(0.5, 0.55, 0.9)]},
 ('h2o', '6'): {'atmpro': 'mls',
  'band': '6',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 5e-25,
  'molecule': 'h2o',
  'ng_adju': [0],
  'ng_refs': [4],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(600, 250)],
  'tsfc': 294,
  'vmax': 1215,
  'vmin': 1100,
  'w_diffuse': [(1.66, 1.66, 1.7, 1.8)],
  'wgt': [(0.3, 0.45, 0.6, 0.95)]},
 ('h2o', '7'): {'atmpro': 'mls',
  'band': '7',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 0,
  'molecule': 'h2o',
  'ng_adju': [0],
  'ng_refs': [7],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(600, 250)],
  'tsfc': 294,
  'vmax': 1380,
  'vmin': 1215,
  'w_diffuse': [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)],
  'wgt': [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]},
 ('h2o', '8'): {'atmpro': 'mls',
  'band': '8',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 0,
  'molecule': 'h2o',
  'ng_adju': [0, -1, 0],
  'ng_refs': [3, 2, 3],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(1, 250), (10, 250), (500, 250)],
  'tsfc': 294,
  'vmax': 1900,
  'vmin': 1380,
  'w_diffuse': [(1.66, 1.66, 1.66), (1.66, 1.66), (1.66, 1.66, 1.66)],
  'wgt': [(0.55, 0.55, 0.85), (0.85, 0.85), (0, 0.3, 0.55)]},
 ('h2o', '9'): {'atmpro': 'mls',
  'band': '9',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 1e-24,
  'molecule': 'h2o',
  'ng_adju': [0],
  'ng_refs': [5],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(500, 250)],
  'tsfc': 294,
  'vmax': 3000,
  'vmin': 1900,
  'w_diffuse': [(1.66, 1.66, 1.66, 1.66, 1.66)],
  'wgt': [(0.4, 0.4, 0.5, 0.6, 0.9)]},
 ('n2o', '7'): {'atmpro': 'mls',
  'band': '7',
  'commitnumber': 'd5931a1',
  'conc': 3.2e-07,
  'dv': 0.001,
  'klin': 2.22e-20,
  'molecule': 'n2o',
  'ng_adju': [0, 0],
  'ng_refs': [2, 2],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(1, 250), (500, 250)],
  'tsfc': 294,
  'vmax': 1380,
  'vmin': 1215,
  'w_diffuse': [(1.8, 1.66), (1.5, 1.8)],
  'wgt': [(0.6, 0.6), (0.7, 0.9)]},
 ('o3', '5'): {'atmpro': 'mls',
  'band': '5',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 2e-20,
  'molecule': 'o3',
  'ng_adju': [0, 0],
  'ng_refs': [2, 5],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(1, 250), (50, 250)],
  'tsfc': 294,
  'vmax': 1100,
  'vmin': 980,
  'w_diffuse': [(1.6, 1.75), (1.55, 1.66, 1.7, 1.75, 1.8)],
  'wgt': [(0.35, 0.6), (0.5, 0.55, 0.7, 0.9, 1.0)]}}

In [16]:

def fpath(param):
    path = os.path.join(pipeline.get_dir_case(param, 
                                               setup=setup_bestfit),
                         'dgdgs.dat')
    return path


def load_dgdgs(path):
    df = pd.read_csv(path, sep=r'\s+')

    df = df.set_index('g')    
    return df



def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4):
    '''
    '''
    strfmt = '{:15.6e}'
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines



def dgdgs_to_F77(dgdgs):
    lines = vector_to_F77list(dgdgs, num_values_per_line=3)
    
    rlines = []
    rlines.append(5 * ' ' + '&/' + lines[0])
    for line in lines[1:-1]:
        rlines.append(5 * ' ' + '&' + line)
    rlines.append(5 * ' ' + '&' + lines[-1] + '/')
    
    fortran = '\n'.join(rlines)
    return fortran


def script():
    for (molecule, band), param in param_map().items():
        dgdgs = load_dgdgs(fpath(param))
        print(molecule, band)
        print(dgdgs_to_F77(dgdgs['dgs']))

In [17]:
script()

co2 3a
     &/   7.177335e-04,   1.596256e-03,   7.011057e-03,
     &   1.205621e-02,   4.641767e-02,   6.328772e-02,
     &   2.638414e-01,   2.288560e-01,   3.762160e-01/
co2 3b
     &/   1.284291e-03,   4.267770e-03,   2.004601e-02,
     &   6.621499e-02,   2.438682e-01,   4.731036e-01,
     &   1.912151e-01/
co2 3c
     &/   9.007998e-04,   1.126758e-03,   6.712162e-03,
     &   1.178262e-02,   4.001021e-02,   7.188866e-02,
     &   3.010401e-01,   2.120142e-01,   3.545245e-01/
co2 9
     &/   3.908974e-04,   3.205781e-03,   2.514517e-02,
     &   5.786103e-02,   1.120021e-01,   8.013950e-01/
h2o 1
     &/   6.371468e-05,   2.196862e-04,   4.132437e-04,
     &   6.498891e-04,   2.968541e-03,   7.563551e-03,
     &   1.371620e-02,   4.421770e-02,   1.123321e-01,
     &   2.686842e-01,   2.959692e-01,   2.532020e-01/
h2o 2
     &/   1.186589e-04,   5.209350e-04,   8.096232e-04,
     &   4.363787e-03,   1.703188e-02,   2.265346e-02,
     &   7.727578e-02,   1.223488e-01,   2.359846e-0